## LLM Pretraining

In [ ]:
#데이터 다운 

import kagglehub  #pip install kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhammaindola/harry-potter-books")

print("Path to dataset files:", path)


### 데이터 텍스트 정리

In [6]:
import re

def clean_text(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        book_text = file.read()
    
    cleaned_text = re.sub(r'\n+', ' ', book_text) # 줄바꿈을 빈칸으로 변경
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # 여러 빈칸을 하나의 빈칸으로
    
    print("cleaned_" + filename, len(cleaned_text), "characters") # 글자 수 출력
    
    
    with open( filename+"_cleaned", 'w', encoding='utf-8') as file:
        file.write(cleaned_text)
    
filenames_list = ["/home/moon/LLM_Study/data/harry-potter-books/versions/1/02 Harry Potter and the Chamber of Secrets.txt"]

for filename in filenames_list:
    clean_text(filename)
        

cleaned_/home/moon/LLM_Study/data/harry-potter-books/versions/1/02 Harry Potter and the Chamber of Secrets.txt 488771 characters


### 토큰 확인

In [9]:
import tiktoken #pip install tiktoken

tokenizer = tiktoken.get_encoding("gpt2")


text = "Harry Potter was a wizard."


tokens = tokenizer.encode(text)

print("글자수:", len(text), "토큰수",len(tokens))

print(tokens)

print(tokenizer.decode(tokens))

for t in tokens:
    print(f"{t}\t -> {tokenizer.decode([t])}")

글자수: 26 토큰수 6
[18308, 14179, 373, 257, 18731, 13]
Harry Potter was a wizard.
18308	 -> Harry
14179	 ->  Potter
373	 ->  was
257	 ->  a
18731	 ->  wizard
13	 -> .


In [13]:
for char in text:
    token_ids = tokenizer.encode(char) # 한 글자씩 인코딩
    decode =tokenizer.decode(token_ids)
    
    print (f"[ {char} ] -> {token_ids} ")

[ 대 ] -> [816] 
[ 사 ] -> [765] 
[ 께 ] -> [1599] 
[ 서 ] -> [712] 
[ 는 ] -> [657] 
[   ] -> [582] 
[ 도 ] -> [720] 
[ ( ] -> [369] 
[ 道 ] -> [26733] 
[ ) ] -> [370] 
[ 를 ] -> [4605] 
[   ] -> [582] 
[ 얻 ] -> [75666] 
[ 은 ] -> [732] 
[   ] -> [582] 
[ 모 ] -> [1679] 
[ 양 ] -> [1509] 
[ 이 ] -> [634] 
[ 구 ] -> [887] 
[ 려 ] -> [1061] 
[ . ] -> [375] 


###  한국어 및 한자 토큰화

In [10]:
from transformers import AutoTokenizer # pip install transformers

tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct")  # KoGPT2 사용
# tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")  # KoGPT2 사용

print("Vocab size :", len(tokenizer))

text = "대사께서는 도(道)를 얻은 모양이구려."

tokens = tokenizer.encode(text)

print(len(text), len(tokens))
print(tokens)
print(tokenizer.decode(tokens))

Vocab size : 102400
21 15
[31980, 1599, 712, 657, 769, 369, 26733, 370, 4605, 4573, 732, 5844, 634, 30556, 375]
대사께서는 도(道)를 얻은 모양이구려.


## 데이터 로더(DataLoader)

TEXT: Harry Potter was a Wizard. <br/>
input(i):  Harry Potter<br/>
target(i+1) was

In [27]:
import torch
from torch.utils.data import Dataset, DataLoader

#pytorch Dataset상속받아서 만드는
class MyDataset(Dataset):
    def __init__(self, txt, max_length, stride):
        self.input_ids = [] 
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt)
        
        print("# of tokens in txt:", len(token_ids))
        
        for i in range(0, len(token_ids) - max_length, stride ):
            input_chunk = token_ids[i:      i + max_length ]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [28]:
# with open("cleaned_한글문서.txt", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
with open("/home/moon/LLM_Study/data/harry-potter-books/versions/1/02 Harry Potter and the Chamber of Secrets.txt_cleaned", 'r', encoding='utf-8-sig') as file: # 선택: -sig를 붙여서 BOM 제거
    txt = file.read()
    
    
dataset = MyDataset(txt, max_length=32, stride=4)

train_loader = DataLoader(dataset, batch_size=128, shuffle=True, drop_last=True)

# of tokens in txt: 123176


In [32]:
dataiter = iter(train_loader)

x, y = next(dataiter)

print(tokenizer.decode(x[0].tolist()))

print("---")

print(tokenizer.decode(y[0].tolist()))

# 한 단어씩 밀려있는 형태!

 they had just been having. “Who’d want to frighten all the Squibs and Muggle-borns out of Hogwart’s?
---
 had just been having. “Who’d want to frighten all the Squibs and Muggle-borns out of Hogwart’s?”


## 뉴럴네트워크 모델 정의

In [38]:
# 모델을 정의할 때 사용하는 상수들

VOCAB_SIZE =  tokenizer.vocab_size  # 50257 tiktoken # 단어가 몇 개 있나

# VOCAB_SIZE = len(toknizer) # AutoTokenizer

CONTEXT_LENGTH = 128  # Shortened context length (orig: 1024)
EMB_DIM = 768  # Embedding dimension
NUM_HEADS = 12  # Number of attention heads
NUM_LAYERS = 12  # Number of layers
DROP_RATE = 0.1  # Dropout rate
QKV_BIAS = False  # Query-key-value bias


In [ ]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        
        assert d_out % NUM_HEADS == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.head_dim = d_out // NUM_HEADS

        self.W_query = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_key = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.W_value = nn.Linear(d_in, d_out, bias=QKV_BIAS)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(DROP_RATE)
        self.register_buffer('mask', torch.triu(torch.ones(CONTEXT_LENGTH, CONTEXT_LENGTH), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, NUM_HEADS, self.head_dim)
        values = values.view(b, num_tokens, NUM_HEADS, self.head_dim)
        queries = queries.view(b, num_tokens, NUM_HEADS, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)

        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(EMB_DIM, 4 * EMB_DIM),
            GELU(),
            nn.Linear(4 * EMB_DIM, EMB_DIM),
        )

    def forward(self, x):
        return self.layers(x)

class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=EMB_DIM,
            d_out=EMB_DIM)
    
        self.ff = FeedForward()
        self.norm1 = LayerNorm(EMB_DIM)
        self.norm2 = LayerNorm(EMB_DIM)
        self.drop_shortcut = nn.Dropout(DROP_RATE)

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


class GPTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.tok_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM)
        self.pos_emb = nn.Embedding(CONTEXT_LENGTH, EMB_DIM)
        self.drop_emb = nn.Dropout(DROP_RATE)

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock() for _ in range(NUM_LAYERS)])

        self.final_norm = LayerNorm(EMB_DIM)
        self.out_head = nn.Linear(EMB_DIM, VOCAB_SIZE, bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits